Neste script treinamos e avaliamos um modelo preditivo para uma tarefa de REGRESSÃO. É possível escolher logo no início o protocolo experimental a ser utilizado: Validação Cruzada (VC) ou Holdout (HO), assim como você pode escolher a técnica de ML entre: KNN ou DT (Decision Tree).

- métricas de avaliação: coeficiente de determinação (R2) e
MAE (mean absolute error). R2 varia entre 0 e 1, sendo 1 ótimo.

Recursos necessários:
- numpy: biblioteca numérica
- sklearn: biblioteca de machine learning, em especial o KNN, DT, as métricas de avaliação e o model_selection que nos permite executar validação cruzada

In [24]:
# Importa bibliotecas necessárias
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn import model_selection
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.datasets import load_diabetes

Carregando a base de dados do problema, representada aqui por X e y, onde:
- X: array contendo N instâncias com M atributos (atributos de entrada do problema)
- y: array contendo o rótulo (atributo alvo) de cada instância em X


In [ ]:
# A base de dados Diabetes disponível ScikitLearn

X, y = load_diabetes(return_X_y=True)
print("Formato de X: ", X.shape)
print("Formato de y: ", y.shape)



Formato de X:  (442, 10)
Formato de y:  (442,)


Neste ponto definimos o protocolo experimental e a técnica de Machine Learning a ser utilizada.

In [100]:
# Defina o protocolo experimental e a técnica de ML
PE='VC' # utilize VC para validação cruzada 10 folds e HO para Holdout 70/30
TE = 'MLP'


match TE:
   case 'KNN':
      reg = KNeighborsRegressor(n_neighbors=3, weights='distance')
      method = 'KNN'
   case 'TREE':
      reg = DecisionTreeRegressor(criterion='squared_error', max_depth=5)
      method = 'Decision Tree'
   case 'MLP':
      reg = MLPRegressor(early_stopping=True, loss='poisson', hidden_layer_sizes=[256,128], max_iter=400, random_state=42)
      method = "Multilayer Perceptron"
   case _: assert False
   

# Instanciando, treinando e testando o modelo conforme o protocolo escolhido

In [101]:
# Treinando e avaliando o modelo de acordo com o protocolo escolhido.
if PE=='HO': # PROTOCOLO Holdout (70/30)
    xtrain, xtest, ytrain, ytest = model_selection.train_test_split(X, y, test_size=0.3, random_state=10) # divide a base em treino e teste
    reg.fit(xtrain, ytrain) # treina o modelo
    result=reg.score(xtest, ytest) # testa o modelo e cálcula R2
    y_pred=reg.predict(xtest) # retorna a predição para cada exemplo de teste
    mae=mean_absolute_error(ytest, y_pred) # calculando MAE
else: # PROTOCOLO Validação Cruzada (10 folds)
    result = model_selection.cross_val_score(reg, X, y, cv=10) # executa validação cruzada considerando 10 folds, calcula R2 médio
    y_pred=model_selection.cross_val_predict(reg, X, y, cv=10) # retorna a predição para cada exemplo de teste.
    mae=mean_absolute_error(y, y_pred) # calculando MAE

Algumas observações sobre a avaliação do modelo:

- A variável *result* que criamos anteriormente já tem R2 médio e desvio padrão (média dos 10 folds na validação cruzada) e apenas um valor no caso de holdout (R2).


In [93]:

# Mostrando R2 conforme o protocolo escolhido
if PE=='HO':
  print ('RESULTADOS DO PROTOCOLO HOLDOUT ->', method)
  print("R2: %.5f" % result)
else: # no caso de validação cruzada - média e desvio padrão
  print ('RESULTADOS DO PROTOCOLO HOLDOUT ->', method)
  print("Mean (R2): %.5f" % result.mean())
  print("Std (R2): %.5f" % result.std())

# Mostrando o erro médio absoluto
print("MAE: %.5f" % mae)

# salvando o modelo
from joblib import dump, load
with open("Modelo.mod", 'wb') as fo:
    dump(reg, fo)

RESULTADOS DO PROTOCOLO HOLDOUT -> Multilayer Perceptron
Mean (R2): 0.42657
Std (R2): 0.12418
MAE: 46.69163
